# generate a target list: see var2df

 'C[30..80] H[40..300] O[10] N[1] P[1]'
 'C[10..40] H[20..100] O[2]'
https://www.lfd.uci.edu/~gohlke/molmass/
https://pypi.org/project/molmass/
https://pyteomics.readthedocs.io/en/latest/mass.html
https://github.com/mobiusklein/brainpy

https://stackoverflow.com/questions/53699012/performant-cartesian-product-cross-join-with-pandas

https://stackoverflow.com/questions/13269890/cartesian-product-in-pandas

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Interval.html

# https://stackoverflow.com/questions/21316628/make-a-pandas-multiindex-from-a-product-of-iterables
# https://stackoverflow.com/questions/44452751/pandas-dataframe-rangeindex
# pd.RangeIndex(start=30, stop=80, step=1)

np.arange(f,t) vs range , no range was faster by 10%
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.from_arrays.html#pandas.MultiIndex.from_arrays

In [8]:
import pandas as pd
import numpy as np
from collections import namedtuple

In [9]:
class MFQL_util():

    mass_of = {
        'C' : 12,
        'H' : 1.0078250321,
        'N' : 14.0030740052,
        'P' : 30.97376151,
        'O' : 15.9949146221,
        'S' : 31.97207069
        }
    
    def __init__(self,elements):
        self.elements = elements
        self._ranges = {}
        self._df = None

    def _makeRanges(self):
        for name, bounds in elements.items():
            self._ranges[name] = np.arange(bounds[0],bounds[1]+1)
    
    def _makeDF(self):
        index  = pd.MultiIndex.from_product(self._ranges.values(), names = self._ranges.keys())
        self._df = pd.DataFrame(index = index).reset_index()
    
    def _cal_M(self):
        self._df['m'] = 0
        for colName in self._df.columns.intersection(test.mass_of.keys()):
            self._df['m'] += self._df[colName] * self.mass_of[colName]

    def _cal_dbr(self):
        x = self._df.get('Cl',0) + self._df.get('Br', 0) + self._df.get('I',0) + self._df.get('F',0)
        self._df['dbr'] = self._df['C'] + 1 - self._df['H']/2 - x/2 + self._df['N']/2

    def _cal_chem(self):
        #https://stackoverflow.com/questions/52673285/performance-of-pandas-apply-vs-np-vectorize-to-create-new-column-from-existing-c
        self._df['chem'] = ''
        for colName in self._df.columns.intersection(test.mass_of.keys()):
            self._df['chem'] += colName + self._df[colName].apply(str) + ' ' 
    
    



In [10]:

elements = {'C':(30,300),'H':(40,300),'O':(10,10),'N':(1,1),'P':(1,1) }
test = MFQL_util(elements)
test._makeRanges()
test._makeDF()
test._cal_M()
test._cal_dbr()
test._cal_chem()
test._df

,C,H,O,N,P,m,dbr,chem
0,30,40,10,1,1,605.238983,11.5,C30 H40 O10 N1 P1
1,30,41,10,1,1,606.246808,11.0,C30 H41 O10 N1 P1
2,30,42,10,1,1,607.254633,10.5,C30 H42 O10 N1 P1
3,30,43,10,1,1,608.262458,10.0,C30 H43 O10 N1 P1
4,30,44,10,1,1,609.270283,9.5,C30 H44 O10 N1 P1
...,...,...,...,...,...,...,...,...
70726,300,296,10,1,1,4103.242191,153.5,C300 H296 O10 N1 P1
70727,300,297,10,1,1,4104.250016,153.0,C300 H297 O10 N1 P1
70728,300,298,10,1,1,4105.257841,152.5,C300 H298 O10 N1 P1
70729,300,299,10,1,1,4106.265666,152.0,C300 H299 O10 N1 P1


# calculate the mass ... efficiently

https://towardsdatascience.com/different-ways-to-iterate-over-rows-in-a-pandas-dataframe-performance-comparison-dc0d5dcef8fe
https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas
https://stackoverflow.com/questions/1422149/what-is-vectorization

note the 'at' operator

# applicable functions
pr.m / fr.m >= 0.25
mark all the entries where this passes

In [28]:
#make fake test set
pr = test._df.sample(n=10, random_state = 1)
fr = test._df.sample(n=3, random_state = 2)

In [29]:
pr.m

58029    3356.919761
25655    1858.841510
59782    3366.340708
9824     1217.545763
49591    2886.246808
2648      763.536334
44393    2668.418959
65910    3768.318837
33685    2169.364184
28874    2090.522288
Name: m, dtype: float64

In [30]:
fr.m

7535     1170.015265
60486    3573.764864
65301    3656.638060
Name: m, dtype: float64

In [41]:
# with comprehension
# (f / p for f in fr.m for p in fr.m )
[(p_idx, f_idx, f / p >=2 ) for p_idx, p in pr.m.items() for f_idx, f in fr.m.items()]

[(58029, 7535, False),
 (58029, 60486, False),
 (58029, 65301, False),
 (25655, 7535, False),
 (25655, 60486, False),
 (25655, 65301, False),
 (59782, 7535, False),
 (59782, 60486, False),
 (59782, 65301, False),
 (9824, 7535, False),
 (9824, 60486, True),
 (9824, 65301, True),
 (49591, 7535, False),
 (49591, 60486, False),
 (49591, 65301, False),
 (2648, 7535, False),
 (2648, 60486, True),
 (2648, 65301, True),
 (44393, 7535, False),
 (44393, 60486, False),
 (44393, 65301, False),
 (65910, 7535, False),
 (65910, 60486, False),
 (65910, 65301, False),
 (33685, 7535, False),
 (33685, 60486, False),
 (33685, 65301, False),
 (28874, 7535, False),
 (28874, 60486, False),
 (28874, 65301, False)]

 https://stackoverflow.com/questions/13269890/cartesian-product-in-pandas
 https://stackoverflow.com/questions/45426843/cartesian-product-of-a-pandas-dataframe-with-itself
 make a dataframe with all the combos and then use .query(string)
 https://jakevdp.github.io/PythonDataScienceHandbook/03.12-performance-eval-and-query.html

no has an isclose / allclose to compare if series are similar

try with query!!!

